## Review

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.listdir()

['Census_BexarCounty_Housing_ACSDP5Y2020.DP04.xlsx',
 'Spreadsheet_MOE&Math.xlsx',
 'README.md',
 'sa2020_stats.ipynb',
 '.gitignore',
 'CMO_CDTractsPrecincts_220406[1].xlsx',
 '.ipynb_checkpoints',
 '.git']

In [97]:
df = pd.read_excel('Spreadsheet_MOE&Math.xlsx')

In [98]:
df = df[:411]

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 10 columns):
Geographic Area Name                                                                          411 non-null object
Council District                                                                              411 non-null object
Estimate!!VEHICLES AVAILABLE!!Occupied housing units                                          411 non-null float64
Margin of Error!!VEHICLES AVAILABLE!!Occupied housing units                                   411 non-null float64
MOE Occupied Housing Units Squared                                                            411 non-null float64
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available                   411 non-null float64
Margin of Error!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available            411 non-null object
MOE No Vehicles Available  Squared                                                           

In [100]:
df.columns = df.columns.str.replace('VEHICLES AVAILABLE', '')
df.columns = df.columns.str.lower().str.replace('\W+','_').str.replace('margin_of_error', 'moe')
df.columns

Index(['geographic_area_name', 'council_district',
       'estimate_occupied_housing_units', 'moe_occupied_housing_units',
       'moe_occupied_housing_units_squared',
       'estimate_occupied_housing_units_no_vehicles_available',
       'moe_occupied_housing_units_no_vehicles_available',
       'moe_no_vehicles_available_squared',
       'percent_occupied_housing_units_no_vehicles_available',
       'percent_moe_occupied_housing_units_no_vehicles_available'],
      dtype='object')

In [101]:
round((df.moe_occupied_housing_units_squared.sum())**.5,1)

5145.0

In [102]:
round((df.moe_no_vehicles_available_squared.sum())**.5,1)

1881.9

### MOE Calculation

moe = 1/y * sqrt( \[moex\]^2 - ( p^2 * \[moey\]^2 ) 

y = total population

x = sample population

p = proportion

#### Example

In [103]:
#total population
y = 466037 + 97360 + 67101
y

630498

In [104]:
#moe of total 
moey = 831
moey

831

In [105]:
#proportion
p = 203_119 / y
p 

0.32215645410453325

In [106]:
#moe of sample
moex = 5070 
moex

5070

In [107]:
#moe = 1/y * sqrt( [moex]^2 - ( p^2 * [moey]^2 ) * 100
def moe_prop(y, moey, p, moex):
    moe = 1/y * (moex**2 - (p**2 * moey**2))**.5 * 100
    return round(moe,2)

In [108]:
moe_prop(y, moey, p, moex)

0.8

### Real Life - Sheet 1

In [109]:
#moe = +/- sqrt(x1^2 + x2^2 + x3^2)
def moe(x):
    moe = (x**2).sum()**.5
    return round(moe,1)

#### moe_prop(y, moey, p, moex) - county

In [110]:
y = df.estimate_occupied_housing_units.sum()
y

709875.0

In [111]:
moey = moe(df.moe_occupied_housing_units)
moey

5145.0

In [112]:
p = df.estimate_occupied_housing_units_no_vehicles_available.sum() / df.estimate_occupied_housing_units.sum()
p

0.07015601338263779

In [113]:
moex = moe(df.moe_occupied_housing_units_no_vehicles_available)
moex

1881.9

In [114]:
moe_prop(y, moey, p, moex)

0.26

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 10 columns):
geographic_area_name                                        411 non-null object
council_district                                            411 non-null object
estimate_occupied_housing_units                             411 non-null float64
moe_occupied_housing_units                                  411 non-null float64
moe_occupied_housing_units_squared                          411 non-null float64
estimate_occupied_housing_units_no_vehicles_available       411 non-null float64
moe_occupied_housing_units_no_vehicles_available            411 non-null object
moe_no_vehicles_available_squared                           411 non-null object
percent_occupied_housing_units_no_vehicles_available        411 non-null object
percent_moe_occupied_housing_units_no_vehicles_available    411 non-null object
dtypes: float64(4), object(6)
memory usage: 32.2+ KB


#### moe_prop(y, moey, p, moex) - city

In [74]:
df1 = df[df.council_district != 0]

In [75]:
y = df1.estimate_occupied_housing_units.sum()
y

572718.0

In [76]:
moey = moe(df1.moe_occupied_housing_units)
moey

4603.9

In [77]:
p = df1.estimate_occupied_housing_units_no_vehicles_available.sum() / df1.estimate_occupied_housing_units.sum()
p

0.0790441369050737

In [78]:
moex = moe(df1.moe_occupied_housing_units_no_vehicles_available)
moex

1799.2

In [79]:
moe_prop(y, moey, p, moex)

0.31

### Real Life - Sheet 2

In [55]:
df2 = pd.read_excel('Spreadsheet_MOE&Math.xlsx', sheet_name='MOE Ratio - P.65')

In [56]:
df2 = df1[:35]

In [57]:
df2.columns = df2.columns.str.replace('VEHICLES AVAILABLE', '')
df2.columns = df2.columns.str.lower().str.replace('\W+','_').str.replace('margin_of_error', 'moe')
df2.columns

Index(['geographic_area_name', 'council_district',
       'estimate_housing_occupancy_total_housing_units',
       'moe_housing_occupancy_total_housing_units', 'total_housing_units_moe',
       'estimate_housing_occupancy_total_housing_units_occupied_housing_units',
       'moe_housing_occupancy_total_housing_units_occupied_housing_units',
       'occupied_housing_units_moe',
       'percent_housing_occupancy_total_housing_units_occupied_housing_units',
       'percent_moe_housing_occupancy_total_housing_units_occupied_housing_units'],
      dtype='object')

#### moe_prop(y, moey, p, moex)

In [58]:
y = df2.estimate_housing_occupancy_total_housing_units.sum()
y

61314.0

In [59]:
moey = moe(df2.moe_housing_occupancy_total_housing_units)
moey

1512.9

In [139]:
test = df2.moe_housing_occupancy_total_housing_units
test = df2.moe_housing_occupancy_total_housing_units**2
test = test.sum()
test = test**.5
test

1512.8833398514241

In [60]:
moex = moe(df2.moe_housing_occupancy_total_housing_units_occupied_housing_units)
moex

1525.1

In [62]:
p = df2.estimate_housing_occupancy_total_housing_units_occupied_housing_units.sum() / df2.estimate_housing_occupancy_total_housing_units.sum()
p

0.8776299050787748

In [63]:
moe_prop(y, moey, p, moex)

1.22